In [1]:
%load_ext autoreload
%autoreload 2

import poisson_atac as patac
import scanpy as sc
import os
import anndata as ad
import wandb
from pytorch_lightning.loggers import WandbLogger

Global seed set to 0


In [2]:
import scvi
scvi.__version__

'0.0.0'

# Read in Neurips data

In [3]:
data_path = '/storage/groups/ml01/workspace/laura.martens/atac_poisson_data/data'

In [4]:
adata = patac.data.load_neurips(data_path, batch=['s1d1', 's1d2'], gex=False, multiome=True, convert_counts=True)

Index(['pct_counts_mt', 'n_counts', 'n_genes', 'size_factors', 'phase',
       'cell_type', 'pseudotime_order_GEX', 'batch', 'pseudotime_order_ATAC',
       'is_train', 'nCount_peaks', 'atac_fragments', 'reads_in_peaks_frac',
       'blacklist_fraction', 'nucleosome_signal', 'size_factor'],
      dtype='object')


In [5]:
adata.var

,feature_types,modality
AL627309.5,GEX,Gene Expression
LINC01409,GEX,Gene Expression
LINC01128,GEX,Gene Expression
NOC2L,GEX,Gene Expression
KLHL17,GEX,Gene Expression
...,...,...
GL000219.1-90062-90937,ATAC,Peaks
GL000219.1-99257-100160,ATAC,Peaks
KI270726.1-27152-28034,ATAC,Peaks
KI270713.1-21434-22336,ATAC,Peaks


In [6]:
adata_mvi = scvi.data.organize_multiome_anndatas(adata)


/home/icb/laura.martens/miniconda3/envs/poisson_atac_new/lib/python3.9/site-packages/anndata/_core/anndata.py:882: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  self._set_dim_df(value, "var")


In [7]:
adata_mvi = adata_mvi[:, adata_mvi.var["modality"].argsort()].copy()
adata_mvi.var

,feature_types,modality
AL627309.5,GEX,Gene Expression
ACTR10,GEX,Gene Expression
PSMA3,GEX,Gene Expression
PSMA3-AS1,GEX,Gene Expression
ARID4A,GEX,Gene Expression
...,...,...
chr15-93164143-93165006,ATAC,Peaks
chr15-93152026-93152897,ATAC,Peaks
chr15-93141085-93141988,ATAC,Peaks
chr15-94206321-94207192,ATAC,Peaks


# Setup model Poisson loss

In [12]:
adata_mvi

AnnData object with n_obs × n_vars = 11685 × 129921
    obs: 'pct_counts_mt', 'n_counts', 'n_genes', 'size_factors', 'phase', 'cell_type', 'pseudotime_order_GEX', 'batch', 'pseudotime_order_ATAC', 'is_train', 'nCount_peaks', 'atac_fragments', 'reads_in_peaks_frac', 'blacklist_fraction', 'nucleosome_signal', 'size_factor', 'modality', '_scvi_batch', '_scvi_labels'
    var: 'feature_types', 'modality'
    uns: '_scvi_uuid', '_scvi_manager_uuid'
    obsm: 'X_pca', 'X_umap', 'gene_activity', 'lsi_full', 'lsi_red', 'umap', '_scvi_extra_categorical_covs'
    layers: 'counts'

In [13]:
patac.model.MULTIVI.setup_anndata(adata_mvi, batch_key='modality',
    size_factor_key = 'n_counts', layer='counts', categorical_covariate_keys=['batch'])

In [14]:
mvi = patac.model.MULTIVI(
    adata_mvi,
    n_genes=(adata_mvi.var['modality']=='Gene Expression').sum(),
    n_regions=(adata_mvi.var['modality']=='Peaks').sum(),
    peak_likelihood='poisson'
)
mvi.view_anndata_setup()

Anndata setup with scvi-tools version 0.0.0.

Setup via `MULTIVI.setup_anndata` with arguments:

{
│   'layer': 'counts',
│   'batch_key': 'modality',
│   'size_factor_key': 'n_counts',
│   'categorical_covariate_keys': ['batch'],
│   'continuous_covariate_keys': None
}

         Summary Statistics          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     Summary Stat Key     ┃ Value  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│         n_cells          │ 11685  │
│          n_vars          │ 129921 │
│         n_batch          │   1    │
│ n_extra_categorical_covs │   1    │
│ n_extra_continuous_covs  │   0    │
└──────────────────────────┴────────┘

                             Data Registry                             
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Registry Key      ┃            scvi-tools Location             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           X            │           adata.layers['counts']           │
│         batch          │          adata.obs['_scvi_batch']          │
│      size_factor       │           adata.obs['n_counts']            │
│ extra_categorical_covs │ adata.obsm['_scvi_extra_categorical_covs'] │
└────────────────────────┴────────────────────────────────────────────┘

                    batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['modality'] │   paired   │          0          │
└───────────────────────┴────────────┴─────────────────────┘

          extra_categorical_covs State Registry          
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃  Source Location   ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['batch'] │    s1d1    │          0          │
│                    │    s1d2    │          1          │
│                    │            │                     │
└────────────────────┴────────────┴─────────────────────┘

In [15]:
mvi.train()

MultiVI Model with INPUTS: n_genes:13431, n_regions:116490
n_hidden: 360, n_latent: 18, n_layers_encoder: 2, n_layers_decoder: 2 , dropout_rate: 0.1, 
latent_distribution: normal, deep injection: False, gene_likelihood: zinb, peak_likelihood: 
poisson
Training status: Not Trained

In [57]:
mvi.get_accessibility_estimates(normalize_regions=True, normalize_cells=True)

,chr3-97974083-97974871,chr3-97971751-97972691,chr3-97957418-97958331,chr3-97871959-97872834,chr3-97954089-97955030,chr3-97952023-97952908,chr3-97910557-97911401,chr3-97882859-97883780,chr3-97991084-97991801,chr3-97823564-97824351,...,chr15-93247444-93248379,chr15-93235883-93236809,chr15-93225467-93226383,chr15-93178749-93179639,chr15-93166693-93167597,chr15-93164143-93165006,chr15-93152026-93152897,chr15-93141085-93141988,chr15-94206321-94207192,KI270713.1-29629-30491
TAGTTGTCACCCTCAC-1-s1d1_paired,0.013851,0.194698,0.012937,0.013869,0.013671,0.014843,0.047327,0.010070,0.013121,0.013768,...,0.015529,0.065064,0.014957,0.025096,0.013518,0.009633,0.034571,0.045475,0.030294,0.015419
CTATGGCCATAACGGG-1-s1d1_paired,0.031771,0.447511,0.029745,0.031835,0.031382,0.034098,0.108509,0.023146,0.030074,0.031578,...,0.035694,0.149754,0.034334,0.057649,0.031033,0.022080,0.079447,0.104416,0.069668,0.035416
CCGCACACAGGTTAAA-1-s1d1_paired,0.016664,0.234074,0.015541,0.016660,0.016445,0.017849,0.056897,0.012098,0.015746,0.016550,...,0.018669,0.078137,0.017969,0.030129,0.016232,0.011572,0.041546,0.054644,0.036393,0.018531
TCATTTGGTAATGGAA-1-s1d1_paired,0.009292,0.130533,0.008666,0.009287,0.009186,0.009969,0.031778,0.006758,0.008781,0.009232,...,0.010422,0.043576,0.010018,0.016804,0.009058,0.006462,0.023170,0.030492,0.020328,0.010356
ACCACATAGGTGTCCA-1-s1d1_paired,0.005465,0.077017,0.005110,0.005467,0.005395,0.005863,0.018667,0.003981,0.005161,0.005425,...,0.006143,0.025725,0.005892,0.009908,0.005320,0.003795,0.013647,0.017935,0.011978,0.006087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CGCGATTCATCATGTG-2-s1d2_paired,0.029860,0.423861,0.028204,0.030207,0.030192,0.031799,0.101810,0.021881,0.028645,0.029882,...,0.034260,0.140508,0.032499,0.053562,0.029257,0.021255,0.075639,0.097654,0.065469,0.033084
AAGCATGAGCATGGTT-2-s1d2_paired,0.031706,0.449976,0.029910,0.032045,0.032067,0.033743,0.108067,0.023194,0.030376,0.031713,...,0.036370,0.148957,0.034474,0.056789,0.031022,0.022537,0.080253,0.103604,0.069428,0.035112
GAGAACCAGCAGGTGG-2-s1d2_paired,0.023737,0.336998,0.022416,0.024019,0.023987,0.025275,0.080886,0.017381,0.022759,0.023745,...,0.027237,0.111683,0.025822,0.042584,0.023231,0.016884,0.060144,0.077546,0.052034,0.026274
AGCTTCCTCCCTGGTT-2-s1d2_paired,0.006996,0.099365,0.006604,0.007078,0.007068,0.007451,0.023855,0.005126,0.006702,0.006993,...,0.008020,0.032900,0.007603,0.012548,0.006849,0.004976,0.017730,0.022883,0.015333,0.007751


# Setup model Bernoulli loss

In [38]:
patac.model.MULTIVI.setup_anndata(adata_mvi, batch_key='modality',
    size_factor_key = 'n_counts', layer='counts', categorical_covariate_keys=['batch'])

In [39]:
mvi = patac.model.MULTIVI(
    adata_mvi,
    n_genes=(adata_mvi.var['modality']=='Gene Expression').sum(),
    n_regions=(adata_mvi.var['modality']=='Peaks').sum(),
    peak_likelihood='bernoulli'
)
mvi.view_anndata_setup()

Anndata setup with scvi-tools version 0.0.0.

Setup via `MULTIVI.setup_anndata` with arguments:

{
│   'layer': 'counts',
│   'batch_key': 'modality',
│   'size_factor_key': 'n_counts',
│   'categorical_covariate_keys': ['batch'],
│   'continuous_covariate_keys': None
}

         Summary Statistics          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     Summary Stat Key     ┃ Value  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│         n_cells          │ 11685  │
│          n_vars          │ 129921 │
│         n_batch          │   1    │
│ n_extra_categorical_covs │   1    │
│ n_extra_continuous_covs  │   0    │
└──────────────────────────┴────────┘

                             Data Registry                             
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Registry Key      ┃            scvi-tools Location             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           X            │           adata.layers['counts']           │
│         batch          │          adata.obs['_scvi_batch']          │
│      size_factor       │           adata.obs['n_counts']            │
│ extra_categorical_covs │ adata.obsm['_scvi_extra_categorical_covs'] │
└────────────────────────┴────────────────────────────────────────────┘

                    batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['modality'] │   paired   │          0          │
└───────────────────────┴────────────┴─────────────────────┘

          extra_categorical_covs State Registry          
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃  Source Location   ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['batch'] │    s1d1    │          0          │
│                    │    s1d2    │          1          │
│                    │            │                     │
└────────────────────┴────────────┴─────────────────────┘

In [40]:
mvi.train()

MultiVI Model with INPUTS: n_genes:13431, n_regions:116490
n_hidden: 360, n_latent: 18, n_layers_encoder: 2, n_layers_decoder: 2 , dropout_rate: 0.1, 
latent_distribution: normal, deep injection: False, gene_likelihood: zinb, peak_likelihood: 
bernoulli
Training status: Not Trained